# Python Notebook for ML Weather Forecasting Project

 Link to data: https://www.ecad.eu/dailydata/

 Project directory: https://drive.google.com/drive/folders/1gnI99uHnB-pbb-bsX1302IQZVNudKgUm?usp=sharing

 ## **Section I: Data retrieval**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import io
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.offsetbox import AnchoredText
from datetime import timedelta

In [ ]:
features_dict = {
    'cloud_cover': 'CC', # cloud cover in oktas
    'glob_rad': 'QQ', # global radiation in W/m2
    'precip': 'RR', # precipitation amount in 0.1 mm
    'mean_temp': 'TG', # mean temperature in 0.1
    'min_temp': 'TN', # minimum temperature in 0.1
    'max_temp': 'TX', # maximum temperature in 0.1
    'wind_speed': 'FG', # wind speed in 0.1 m/s
    'humidity': 'HU', # humidity in 1 %
    'sea_level_pressure': 'PP', # sea level pressure in 0.1 hPa
    'sunshine': 'SS' # sunshine in 0.1 Hours
}

city = 'Valencia'
features = ['glob_rad',
            'precip',
            'mean_temp',
            'min_temp',
            'max_temp',
            'cloud_cover',
            'wind_speed',
            'humidity',
            'sea_level_pressure',
            'sunshine']

project_dir = 'drive/MyDrive/ECIU ML Project Submission/Colab Notebooks/'

In [ ]:
new_import = False

if new_import:

  total_data = pd.DataFrame(columns=['DATE'])

  for i, feat in enumerate(features):
    feat_ID = features_dict[feat]
    files_list = os.listdir(project_dir + city)
    file_name = [f for f in files_list if feat_ID in f][0]
    f = open(project_dir + city + '/' + file_name,'r')
    for line in f:
      if 'SOUID,    DATE,' not in line:
        pass
      else: # this is where the data of the .txt file starts
        header = line
        data = f.readlines()[:] # .readlines() starts from the current index
        data = pd.read_csv(io.StringIO(''.join(data)), delimiter=',',
                          header=None,
                          names=header[:-1].replace(' ','').split(','))
        # source and station ID not needed
        data = data.drop(['SOUID', 'STAID'], axis=1, errors='ignore')
        data['DATE'] = pd.to_datetime(data['DATE'], format='%Y%m%d')
        # set measurements to NaN where quality ID == 9
        data.loc[data['Q_'+feat_ID] == 9, feat_ID] = np.nan
        data = data.drop(['Q_'+feat_ID], axis=1) # quality ID not needed anymore
        data = data.rename(columns={feat_ID: feat})
        data = data.set_index('DATE') # setting date as index of df
        data = data.resample('W').mean()
        # index of the total dataframe is set to the index of the 1st feature
        if i == 0:
          total_data['DATE'] = data.index
          total_data = total_data.set_index('DATE')
        total_data = pd.concat([total_data, data], axis=1, join='inner')
        break

  # saving data to csv
  # total_data.to_csv(project_dir+city+'/TOTAL.csv')

##**Section II: Data visualization and feature selection**

In [ ]:
total_data = pd.read_csv(project_dir+city+'/TOTAL.csv')
total_data['DATE'] = pd.to_datetime(total_data['DATE'])
total_data = total_data.set_index('DATE')
# dropping wind speed due to its incompleteness
total_data = total_data.drop(['wind_speed'], axis=1)

# the data from around 1955 to 2003 seems to be most complete
start = '01-01-1995'
end = '01-01-2003'
sub_data = total_data[start:end]

N = 3
pred_target = 'mean_temp'
shifted_data = pd.DataFrame(sub_data[pred_target])

def derive_nth_day_feature(original_df, shifted_df, feature, N):
    days = original_df.shape[0]
    nth_prior_measurements = [np.nan]*N + [
      original_df[feature][i-N] for i in range(N, days)]
    col_name = "{}_{}".format(feature, N)
    shifted_df[col_name] = nth_prior_measurements

for feat in sub_data.columns:
  for n in range(1,N+1):
    shifted_df = derive_nth_day_feature(original_df=sub_data,
                                        shifted_df=shifted_data,feature=feat,
                                        N=n)

# drop NaN values that were created due to shifting
shifted_data = shifted_data.dropna()
# shifted_data.info()

In [ ]:
# plot data histograms

for i, col in enumerate(shifted_data.columns):
  plt.figure(i, figsize=(12, 6), dpi=80)
  shifted_data[col].hist(bins=20)
  plt.title('Distribution of %s' % col)
  plt.xlabel(col)
  plt.show()

Calculating the Pearson correlation coefficient (measure for linear correlation between two variables):

$$r_{xy}={\frac {\sum _{i=1}^{m}(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})}{{\sqrt {\sum _{i=1}^{m}(x_{i}-{\bar {x}})^{2}}}{\sqrt {\sum _{i=1}^{m}(y_{i}-{\bar {y}})^{2}}}}}$$

In [ ]:
corr_coeffs = shifted_data.corr()[['mean_temp']].sort_values('mean_temp')
print(corr_coeffs)

In [ ]:
# removing features with |r| < 0.3
to_remove = ('humidity', 'precip', 'sea_level_pressure', 'cloud_cover')

# predictors will be all the other features
predictors = [f for f in shifted_data.columns if not f.startswith(to_remove)]
predictors.remove(pred_target)
predictor_data = shifted_data[predictors]
target_data = shifted_data[pred_target]

In [ ]:
# plot correlation diagrams

Ncols = N
Nrows = len(predictors)//N
# manually set the parameters of the figure to and appropriate size
plt.rcParams['figure.figsize'] = [16, 20]
plt.rcParams.update({'font.size': 22})


# call subplots specifying the grid structure
fig, axes = plt.subplots(nrows=Nrows, ncols=Ncols, sharey=True)

# rearrange data into a 2D array
arr = np.array(predictors).reshape(Nrows, Ncols)

# use enumerate to loop over the arr 2D array of rows and columns
# and create scatter plots of each mean_temp vs each feature
for row, col_arr in enumerate(arr):
    for col, feat in enumerate(col_arr):
        axes[row, col].scatter(shifted_data[feat], shifted_data[pred_target])
        axes[row, col].grid(which='both')
        anchored_text = AnchoredText('$r=%.2f$'%corr_coeffs[pred_target][feat],
                                     loc='lower right')
        axes[row, col].add_artist(anchored_text)
        if col == 0:
            axes[row, col].set(xlabel=feat, ylabel=pred_target)
            axes[row, col].set_xticks([])
            axes[row, col].set_yticks([])
        else:
            axes[row, col].set(xlabel=feat)
            axes[row, col].set_xticks([])
            axes[row, col].set_yticks([])
plt.tight_layout(pad=0.6)
plt.show()

##**Section III: Linear Regression**

Cost function:

$$J(\mathbf{\theta})=\frac{1}{2m}\sum_{i=1}^{m}\bigl[h_{\theta}(\mathbf{x}_i)-y_i\bigr]^2$$

In [ ]:
def computeCost(X, y, theta):
    """
    Take the numpy arrays X, y, theta and return the cost function J for this
    theta.

    """
    m = len(y)
    h = X.dot(theta)
    square_err = (h - y) ** 2
    J = 1 / (2 * m) * sum(square_err)

    return J

Gradient descent:
$\mathbf{\theta}^{(k+1)} = \mathbf{\theta}^{(k)} - \alpha\nabla_{\theta}J(\mathbf{\theta}^{(k)}) = \mathbf{\theta}^{(k)} - \alpha\frac{1}{m} \mathbf{X}^
\mathsf{T}(\mathbf{h}_{\mathbf{\theta}}^{(k)}-\mathbf{y})$

where

$\mathbf{h}_{\mathbf{\theta}} = \begin{bmatrix}h_{\mathbf{\theta}}(\mathbf{x}_1)\\h_{\mathbf{\theta}}(\mathbf{x}_2)\\\cdots\\h_{\mathbf{\theta}}(\mathbf{x}_m)\end{bmatrix}$

In [ ]:
def gradientDescent(X, y, theta, alpha, num_iters):
    """
    Take numpy arrays X, y and theta and update theta by taking num_iters
    gradient steps with learning rate alpha

    Return: theta and the list of the cost of theta (J_history) during each
    iteration
    """

    m = len(y)
    J_history = []

    for k in range(num_iters):
      h = X.dot(theta)
      # vectorized way to compute all gradients simultaneously
      grad = X.transpose().dot(h - y)
      descent = (alpha / m) * grad
      theta = theta - descent
      J_history.append(computeCost(X, y, theta))

    return theta, J_history

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

if 'intercept' not in predictor_data.columns:
  predictor_data.insert(0, 'intercept', 1)

normalized = False

if not normalized:
  X_train, X_test, y_train, y_test = train_test_split(predictor_data,
                                                    target_data,
                                                    test_size=0.2,
                                                    random_state=5)
else: # option to use feature normalization
  scaler = StandardScaler()
  normalized_data = pd.DataFrame(
      scaler.fit_transform(predictor_data, target_data),
      index = predictor_data.index,
      columns = predictor_data.columns.delete(0))

  X_train, X_test, y_train, y_test = train_test_split(normalized_data,
                                                    target_data,
                                                    test_size=0.2,
                                                    random_state=5)

In [ ]:
m = X_train.shape[0]
n = X_train.shape[1]

# initialize the fitting parameters theta to 0 (np.zeros)

theta = np.zeros((n, 1))

theta, J_history = gradientDescent(X=X_train.values,
                                   y=y_train.values.reshape(m, 1),
                                   theta=theta, alpha=1e-7, num_iters=100)

In [ ]:
# print model function h(x)

print('h(x) = %.2f' % theta[0])
for i, pred in enumerate(predictors):
  print('\t+ %.2f * %s' % (theta[i+1], pred))

In [ ]:
# plot convergence of cost function

plt.figure(figsize=(12, 6))
plt.rcParams.update({'font.size': 22})
plt.xlabel('Number of iterations $k$')
plt.ylabel('Cost function $J(theta)$')
plt.grid('both')
plt.plot(J_history)

In [ ]:
# compute predictions for y_test using the model

predictions = X_test.dot(theta).values

# Evaluate the prediction performance of the model
from sklearn.metrics import mean_squared_error, median_absolute_error

MSE = 0.1 * mean_squared_error(y_test.values, predictions.ravel())
MAE = 0.1 * median_absolute_error(y_test.values, predictions.ravel())

print("The Mean Squared Error: %.2f degrees celsius" % MSE)
print("The Median Absolute Error: %.2f degrees celsius" % MAE)

The Mean Squared Error: 56.88 degrees celsius
The Median Absolute Error: 1.49 degrees celsius


In [ ]:
# plot distribution of absolute prediction error

plt.figure(figsize=(12, 6))
plt.rcParams.update({'font.size': 22})
plt.xlabel('Absolute Error in °C')
plt.ylabel('# of predictions')
plt.hist(0.1*abs(y_test.values - predictions.ravel()), bins=20)

In [ ]:
# plot correlation diagrams now including the predictions

Ncols = N
Nrows = len(predictors)//N
# manually set the parameters of the figure to and appropriate size
plt.rcParams['figure.figsize'] = [16, 22]

# call subplots specifying the grid structure we desire and that
# the y axes should be shared
fig, axes = plt.subplots(nrows=Nrows, ncols=Ncols, sharey=True)

# rearrange predictor names into 2D array
arr = np.array(predictors).reshape(Nrows, Ncols)

# create dataframe that includes linspaces for all predictors from min to max
min_max_df = pd.DataFrame()
min_max_df['intercept'] = np.ones(50)
for pred in predictors:
  min_max_df[pred] = np.linspace(min(predictor_data[pred]),
                                 max(predictor_data[pred]))

predictions = min_max_df.dot(theta).values

# loop over 2D array and create scatter plots of mean_temp vs each feature
for row, col_arr in enumerate(arr):
    for col, feat in enumerate(col_arr):
        axes[row, col].scatter(predictor_data[feat], target_data)
        axes[row, col].plot(min_max_df[feat], predictions, color='r')
        if col == 0:
            axes[row, col].set(xlabel=feat, ylabel=pred_target)
        else:
            axes[row, col].set(xlabel=feat)
plt.tight_layout(pad=0.6)
plt.show()

## **Section IV: Neural Network**

In [ ]:
import tensorflow as tf
from sklearn.metrics import explained_variance_score, mean_squared_error, \
median_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
# split data into training set and a temporary set
X_train, X_tmp, y_train, y_tmp = train_test_split(shifted_data,
                                                  target_data,
                                                  test_size=0.2,
                                                  random_state=5)

# take the remaining 20% of data in X_tmp, y_tmp and split them evenly
X_test, X_val, y_test, y_val = train_test_split(X_tmp, y_tmp, test_size=0.5,
                                                random_state=5)

print("Training instances   {}, Training features   {}".format(
    X_train.shape[0], X_train.shape[1]))
print("Validation instances {}, Validation features {}".format(
    X_val.shape[0], X_val.shape[1]))
print("Testing instances    {}, Testing features    {}".format(
    X_test.shape[0], X_test.shape[1]))

In [ ]:
# setting up the ANN
feature_cols = [tf.feature_column.numeric_column(col) for col in
                predictor_data.columns]
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                      hidden_units=[50, 50],
                                      model_dir=project_dir+'ANN_data')

In [ ]:
def input_fn(X, y=None, num_epochs=None, shuffle=True, batch_size=1000):
  # we have to drop the datetime index by adding .reset_index(drop=True)
  X = X.reset_index(drop=True)
  if y is not None:
    y = y.reset_index(drop=True)
  return tf.compat.v1.estimator.inputs.pandas_input_fn(x=X, y=y,
                                                       num_epochs=num_epochs,
                                                       shuffle=shuffle,
                                                       batch_size=batch_size)

In [ ]:
# training of the ANN; a lot of output will be printed in the console!

evaluations = []
STEPS = 100
for i in range(50):
  print('\nIteration: ' + str(i+1))
  regressor.train(input_fn=input_fn(X_train, y=y_train,
                                    batch_size=X_train.shape[0]//2),
                  steps=STEPS)
  evaluations.append(regressor.evaluate(input_fn=input_fn(X_val,
                                                          y_val,
                                                          num_epochs=1,
                                                          shuffle=False)))

In [ ]:
# manually set the parameters of the figure
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams.update({'font.size': 22})

loss_values = [ev['average_loss'] for ev in evaluations]
training_steps = [ev['global_step'] for ev in evaluations]

# plot convergence of the total squared error
plt.plot(training_steps, loss_values)
plt.xlabel('Training steps (Epochs = steps / 2)')
plt.ylabel('Total squared error in °C')
plt.grid()
plt.show()

In [ ]:
# use ANN to predict mean temperatures
pred = regressor.predict(input_fn=input_fn(X_test, num_epochs=1, shuffle=False))
predictions = np.array([p['predictions'][0] for p in pred])

MSE = 0.1 * mean_squared_error(y_test.values, predictions.ravel())
MAE = 0.1 * median_absolute_error(y_test.values, predictions.ravel())

print("\nThe Mean Squared Error: %.2f degrees Celcius" % MSE)
print("The Median Absolute Error: %.2f degrees Celcius" % MAE)

In [ ]:
# plot distribution of absolute prediction error
plt.figure(figsize=(12, 6))
plt.rcParams.update({'font.size': 22})
plt.xlabel('Absolute Error in °C')
plt.ylabel('# of predictions')
plt.hist(0.1*abs(y_test.values - predictions.ravel()), bins=20)